In [ ]:
# Importing required libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
import matplotlib.pyplot as plt

# Configuring memory growth for GPUs
# Keep GPU from using up all memory
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Loading and preprocessing training data
data = tf.keras.utils.image_dataset_from_directory("train")
data = data.map(lambda x, y: (x / 255, y))  # Scaling the image data between 0 and 1

# Splitting data into training and validation sets
train_size = int(len(data) * 0.8) # Train 80% of data
val_size = int(len(data) * 0.2) + 1 # Leave 20% to validate(test)
train_data = data.take(train_size) 
val_data = data.skip(train_size).take(val_size) #validation data does not overlap training

# Checking minimum value of the first batch of data
data.as_numpy_iterator().next()[0].min()

# Obtaining a batch of data for visualization
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()

# Creating subplots for visualizing the batch of images
fig, ax = plt.subplots(ncols=4, figsize=(20, 20)) # four plots in a single row 20X20 inches
for idx, img in enumerate(batch[0][:4]): # sets up loop for first 4 images in the batch array
    ax[idx].imshow(img) # Shows images
    ax[idx].title.set_text(batch[1][idx]) # Gives title

# Loading and preprocessing test data
test_dir = tf.keras.utils.image_dataset_from_directory("test") # Gets images from "test" folder
test_data = test_dir.map(lambda x, y: (x / 255, y)) # Scale images between 0 & 1

# Configuring AUTOTUNE for performance optimization of data
AUTOTUNE = tf.data.experimental.AUTOTUNE

# Caching and prefetching the training, validation, and test data
# Minimizing I/O latency and maximizing GPU or CPU utilization
train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
val_data = val_data.cache().prefetch(buffer_size=AUTOTUNE)
test_data = test_data.cache().prefetch(buffer_size=AUTOTUNE)

# Creating the model architecture
model = Sequential([
    Conv2D(16, (3, 3), 1, activation='relu', input_shape=(256, 256, 3)), #creates a 2D convolutional laye
    MaxPooling2D(), #reduces the spatial dimensions of the input
    Conv2D(32, (3, 3), 1, activation='relu'),
    MaxPooling2D(),
    Conv2D(16, (3, 3), 1, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compiling the model
model.compile(optimizer="adam", loss=tf.losses.BinaryCrossentropy(), metrics=["accuracy"]) 

# Training the model
history = model.fit(train_data, epochs=20, validation_data=val_data)

# Evaluating the model on test data
test_loss, test_accuracy = model.evaluate(test_data)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Visualizing the evaluation of the test model
evaluation_results = model.evaluate(test_data)
labels = ['Test Loss', 'Test Accuracy']
values = [evaluation_results[0], evaluation_results[1]]

plt.plot(labels, values, marker='o', color='teal')
plt.xlabel('Metrics')
plt.ylabel('Value')
plt.title('Evaluation of Test Model')
plt.show()

# Visualizing the training history
fig, axes = plt.subplots(2, figsize=(10, 8)) # 2 plots 10X8 inches
fig.suptitle("Training History", fontsize=20) # Title for both 

# First plot
axes[0].plot(history.history["loss"], color='teal', label='loss') # plots loss with color and label
axes[0].plot(history.history["val_loss"], color='orange', label='val_loss') # plot val_loss with color and label
axes[0].set_xlabel("Epochs") # Vertical label name
axes[0].set_ylabel("Loss") # Horizontal label name 
axes[0].legend(loc="upper right") # location of label

# Second plot
axes[1].plot(history.history["accuracy"], color='teal', label='accuracy') # plot accuracy with color and label
axes[1].plot(history.history["val_accuracy"], color='orange', label='val_accuracy') #plot val_accuracy with color and label
axes[1].set_xlabel("Epochs") # Vertical label name
axes[1].set_ylabel("Accuracy") # Horizontal label name 
axes[1].legend(loc="lower right") # location of label

# Displaying the plots
plt.tight_layout()
plt.show()